In [13]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install graphviz


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [18]:
import json
import psycopg2
import graphviz
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout

In [19]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    database="TPC-H",
    user="postgres",
    password="postgres123",
    host="localhost",
    port="5432"
)

In [20]:
# Create a cursor
cursor = conn.cursor()

In [21]:

# Specify the SQL query you want to explain
#input_query = "SELECT * FROM region" 

input_query = """
    select 
      l_returnflag,
      l_linestatus,
      sum(l_quantity) as sum_qty,
      sum(l_extendedprice) as sum_base_price,
      sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
      sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
      avg(l_quantity) as avg_qty,
      avg(l_extendedprice) as avg_price,
      avg(l_discount) as avg_disc,
      count(*) as count_order
    from
      lineitem
    where
      l_extendedprice > 100
    group by
      l_returnflag,
      l_linestatus
    order by
      l_returnflag,
      l_linestatus;
"""

In [22]:
cursor.execute(f"EXPLAIN (FORMAT JSON) {input_query}")
plan = cursor.fetchall()
query_plan_dict: dict = plan[0][0][0]["Plan"] #query plan is nested structure, needa extract innermost Plan


In [24]:
plan

[([{'Plan': {'Node Type': 'Aggregate',
     'Strategy': 'Sorted',
     'Partial Mode': 'Finalize',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 232383.52,
     'Total Cost': 232385.48,
     'Plan Rows': 6,
     'Plan Width': 236,
     'Group Key': ['l_returnflag', 'l_linestatus'],
     'Plans': [{'Node Type': 'Gather Merge',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Async Capable': False,
       'Startup Cost': 232383.52,
       'Total Cost': 232384.92,
       'Plan Rows': 12,
       'Plan Width': 236,
       'Workers Planned': 2,
       'Plans': [{'Node Type': 'Sort',
         'Parent Relationship': 'Outer',
         'Parallel Aware': False,
         'Async Capable': False,
         'Startup Cost': 231383.5,
         'Total Cost': 231383.52,
         'Plan Rows': 6,
         'Plan Width': 236,
         'Sort Key': ['l_returnflag', 'l_linestatus'],
         'Plans': [{'Node Type': 'Aggregate',
           'Strategy'

In [23]:
query_plan_dict

{'Node Type': 'Aggregate',
 'Strategy': 'Sorted',
 'Partial Mode': 'Finalize',
 'Parallel Aware': False,
 'Async Capable': False,
 'Startup Cost': 232383.52,
 'Total Cost': 232385.48,
 'Plan Rows': 6,
 'Plan Width': 236,
 'Group Key': ['l_returnflag', 'l_linestatus'],
 'Plans': [{'Node Type': 'Gather Merge',
   'Parent Relationship': 'Outer',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 232383.52,
   'Total Cost': 232384.92,
   'Plan Rows': 12,
   'Plan Width': 236,
   'Workers Planned': 2,
   'Plans': [{'Node Type': 'Sort',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 231383.5,
     'Total Cost': 231383.52,
     'Plan Rows': 6,
     'Plan Width': 236,
     'Sort Key': ['l_returnflag', 'l_linestatus'],
     'Plans': [{'Node Type': 'Aggregate',
       'Strategy': 'Hashed',
       'Partial Mode': 'Partial',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Asy

HERE ONWARDS is Old Testing

In [211]:
# Execute the EXPLAIN command for the specified query and fetch the result as JSON
cursor.execute(f"EXPLAIN (ANALYSE, COSTS, BUFFERS, TIMING, FORMAT JSON) {input_query}")
qep_json = cursor.fetchone()[0]
plan = cursor.fetchall()

In [217]:
# Execute the EXPLAIN command for the specified query and fetch the result as JSON
cursor.execute(f"EXPLAIN (ANALYSE, COSTS, BUFFERS, TIMING, FORMAT JSON) {input_query}")
plan = cursor.fetchall()
if not plan:
    print("No rows returned by the query")
else:
    for row in plan:
        qep_json1 = row[0]
        # Process qep_json as needed

In [224]:
qep_json1

[{'Plan': {'Node Type': 'Sort',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 157195.79,
   'Total Cost': 157195.79,
   'Plan Rows': 1,
   'Plan Width': 34,
   'Actual Startup Time': 928.48,
   'Actual Total Time': 937.938,
   'Actual Rows': 411,
   'Actual Loops': 1,
   'Sort Key': ['(count(*)) DESC', 's.s_name'],
   'Sort Method': 'quicksort',
   'Sort Space Used': 50,
   'Sort Space Type': 'Memory',
   'Shared Hit Blocks': 1078254,
   'Shared Read Blocks': 150200,
   'Shared Dirtied Blocks': 0,
   'Shared Written Blocks': 0,
   'Local Hit Blocks': 0,
   'Local Read Blocks': 0,
   'Local Dirtied Blocks': 0,
   'Local Written Blocks': 0,
   'Temp Read Blocks': 0,
   'Temp Written Blocks': 0,
   'Plans': [{'Node Type': 'Aggregate',
     'Strategy': 'Sorted',
     'Partial Mode': 'Simple',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 157195.76,
     'Total Cost': 157195.78,
     'Plan Row

In [221]:
qep_json

[{'Plan': {'Node Type': 'Sort',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 157195.79,
   'Total Cost': 157195.79,
   'Plan Rows': 1,
   'Plan Width': 34,
   'Actual Startup Time': 862.243,
   'Actual Total Time': 870.643,
   'Actual Rows': 411,
   'Actual Loops': 1,
   'Sort Key': ['(count(*)) DESC', 's.s_name'],
   'Sort Method': 'quicksort',
   'Sort Space Used': 50,
   'Sort Space Type': 'Memory',
   'Shared Hit Blocks': 1078258,
   'Shared Read Blocks': 150196,
   'Shared Dirtied Blocks': 0,
   'Shared Written Blocks': 0,
   'Local Hit Blocks': 0,
   'Local Read Blocks': 0,
   'Local Dirtied Blocks': 0,
   'Local Written Blocks': 0,
   'Temp Read Blocks': 0,
   'Temp Written Blocks': 0,
   'Plans': [{'Node Type': 'Aggregate',
     'Strategy': 'Sorted',
     'Partial Mode': 'Simple',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 157195.76,
     'Total Cost': 157195.78,
     'Plan Ro

In [193]:
qep_json[0]['Plan']['Plans']

[{'Node Type': 'Aggregate',
  'Strategy': 'Sorted',
  'Partial Mode': 'Simple',
  'Parent Relationship': 'Outer',
  'Parallel Aware': False,
  'Async Capable': False,
  'Startup Cost': 157195.76,
  'Total Cost': 157195.78,
  'Plan Rows': 1,
  'Plan Width': 34,
  'Actual Startup Time': 844.888,
  'Actual Total Time': 854.646,
  'Actual Rows': 411,
  'Actual Loops': 1,
  'Group Key': ['s.s_name'],
  'Shared Hit Blocks': 1081361,
  'Shared Read Blocks': 147093,
  'Shared Dirtied Blocks': 0,
  'Shared Written Blocks': 0,
  'Local Hit Blocks': 0,
  'Local Read Blocks': 0,
  'Local Dirtied Blocks': 0,
  'Local Written Blocks': 0,
  'Temp Read Blocks': 0,
  'Temp Written Blocks': 0,
  'Plans': [{'Node Type': 'Sort',
    'Parent Relationship': 'Outer',
    'Parallel Aware': False,
    'Async Capable': False,
    'Startup Cost': 157195.76,
    'Total Cost': 157195.76,
    'Plan Rows': 1,
    'Plan Width': 26,
    'Actual Startup Time': 844.877,
    'Actual Total Time': 854.079,
    'Actual Rows

In [182]:
#function to visualise qep and recursievly build qep tree

def build_graph(plan, G=None):
    if G is None:
        G = nx.DiGraph()

    # Add node to the graph
    G.add_node(plan['Node Type'])

    # Recursively build the graph for child plans
    if 'Plans' in plan:
        for child_plan in plan['Plans']:
            G.add_edge(plan['Node Type'], child_plan['Node Type'])
            build_graph(child_plan, G)

In [175]:
# Create a directed graph
graph = nx.DiGraph()

In [191]:
# Build the graph for the QEP JSON
build_graph(qep_json[0]['Plan'], graph)

In [195]:
# Arrange the graph hierarchically using graphviz_layout
pos = graphviz_layout(graph, prog='dot')

# Visualize the graph
plt.figure(figsize=(10, 8))
nx.draw(graph, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=10, font_color="black", font_weight="bold", arrowsize=20, linewidths=0, edgecolors="black")
plt.show()

C:\Users\User\AppData\Local\Temp\ipykernel_36796\1334485859.py:2: DeprecationWarning: nx.nx_pydot.graphviz_layout depends on the pydot package, which hasknown issues and is not actively maintained. Consider usingnx.nx_agraph.graphviz_layout instead.

See https://github.com/networkx/networkx/issues/5723
  pos = graphviz_layout(graph, prog='dot')


FileNotFoundError: [WinError 2] "dot" not found in path.

In [84]:
# Execute the EXPLAIN command for the specified query and fetch the result as JSON
cursor.execute(f"EXPLAIN (BUFFERS, FORMAT JSON) {input_query}")
qep_buffers_json = cursor.fetchone()[0]

In [85]:
qep_buffers_json

[{'Plan': {'Node Type': 'Sort',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 157195.79,
   'Total Cost': 157195.79,
   'Plan Rows': 1,
   'Plan Width': 34,
   'Sort Key': ['(count(*)) DESC', 's.s_name'],
   'Shared Hit Blocks': 0,
   'Shared Read Blocks': 0,
   'Shared Dirtied Blocks': 0,
   'Shared Written Blocks': 0,
   'Local Hit Blocks': 0,
   'Local Read Blocks': 0,
   'Local Dirtied Blocks': 0,
   'Local Written Blocks': 0,
   'Temp Read Blocks': 0,
   'Temp Written Blocks': 0,
   'Plans': [{'Node Type': 'Aggregate',
     'Strategy': 'Sorted',
     'Partial Mode': 'Simple',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 157195.76,
     'Total Cost': 157195.78,
     'Plan Rows': 1,
     'Plan Width': 34,
     'Group Key': ['s.s_name'],
     'Shared Hit Blocks': 0,
     'Shared Read Blocks': 0,
     'Shared Dirtied Blocks': 0,
     'Shared Written Blocks': 0,
     'Local Hit Blocks': 

In [91]:

# Close the cursor and connection
cursor.close()
conn.close()